# Six-bar mechanism balancing

###### Luis Eduardo Robles Jiménez

With this work, I pretend to find out the behavior of the function with different tools, in order to understand what the most important variables are, find patterns, identify overlapping zones of minima between the functions we're optimizing, graphing data, etc.

In [1]:
from BetaShF import *
from BetaShM import *
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import plotly.express as px
from sympy import *
import numpy as np
import scipy as sp
import time


assert ShF([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) == 1.0
assert ShM([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) == 1.0

#### Constraints

$$ -0.16m \leq x_{cn}, y_{cn} \leq 0.16m $$

$$ 0.005m \leq t_{cn} \leq 0.04m $$

#### Objective function

In [ ]:
def objective(s, ShF, ShM):
    return 0.5*ShF(s) + 0.5*ShM(s)

## Mathematical analysis 

### Sympy

In [3]:
from momentFormula import *
from forceFormula import *

In [ ]:
a = srepr(forceF)
b = srepr(momentF)

## Sample generation

In [2]:
nSamples, nVariables = 500, 15
nWeights = nVariables // 3
samples = np.zeros((nSamples, nVariables))
bounds = np.zeros((nVariables, 2))
fitness = np.zeros((nSamples))
shForces = np.zeros((nSamples))
shMoments = np.zeros((nSamples))

### Differential Evolution

In [ ]:
for i in range(nVariables):
    if (i + 1) % 3 == 0: samples[:, i] = np.random.uniform(0.005, 0.04, (nSamples))
    else: samples[:, i] = np.random.uniform(-0.16, 0.16, (nSamples))
for i in range(nSamples):
    fitness[i] = objective(samples[i], ShF, ShM)
"""
for v in range(nVariables):
    bounds[v] = [0.005, 0.04] if (v + 1) % 3 == 0 else [-0.16, 0.16]
worst = 0.0
for s in range(nSamples):
    print("\rIteration:", s, "\tWorst fitness:", worst, end = "")
    p = False
    if s < nSamples // 10: p = True
    r = sp.optimize.differential_evolution(objective, bounds, args = (ShF, ShM), maxiter = 25, popsize = 10, polish = p)
    samples[s] = r.x
    fitness[s] = r.fun
    worst = np.max([worst, fitness[s]])
"""

### Pymoo

In [4]:
from pymoo.core.problem import ElementwiseProblem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.termination import get_termination
from pymoo.optimize import minimize

In [5]:
class MyProblem(ElementwiseProblem):

    def __init__(self):
        super().__init__(n_var=nVariables,
                         n_obj=2,
                         n_ieq_constr=0,
                         xl=np.array([-0.16, -0.16, 0.005, -0.16, -0.16, 0.005, -0.16, -0.16, 0.005, -0.16, -0.16, 0.005, -0.16, -0.16, 0.005]),
                         xu=np.array([0.16, 0.16, 0.04, 0.16, 0.16, 0.04, 0.16, 0.16, 0.04, 0.16, 0.16, 0.04, 0.16, 0.16, 0.04]))

    def _evaluate(self, x, out, *args, **kwargs):
        f1 = ShF(x)
        f2 = ShM(x)

        out["F"] = [f1, f2]
        out["G"] = []


problem = MyProblem()

In [16]:
algorithm = NSGA2(
    pop_size=100,
    n_offsprings=30,
    sampling=FloatRandomSampling(),
    crossover=SBX(prob=0.9, eta=15),
    mutation=PM(eta=20),
    eliminate_duplicates=True
)

In [17]:
termination = get_termination("n_gen", 100)

In [18]:
res = minimize(problem,
               algorithm,
               termination,
               seed=1,
               save_history=True,
               verbose=True)

X = res.X
F = res.F

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |      100 |      2 |             - |             -
     2 |      130 |      2 |  0.6504481377 |         ideal
     3 |      160 |      4 |  0.2504652072 |         ideal
     4 |      190 |      2 |  0.4448574325 |         ideal
     5 |      220 |      2 |  0.000000E+00 |             f
     6 |      250 |      2 |  0.0056416747 |         ideal
     7 |      280 |      2 |  0.000000E+00 |             f
     8 |      310 |      4 |  0.0589758464 |         ideal
     9 |      340 |      4 |  0.0266405083 |             f
    10 |      370 |      6 |  0.4765720928 |         ideal
    11 |      400 |      7 |  0.0301979005 |         nadir
    12 |      430 |      9 |  0.0555568624 |         ideal
    13 |      460 |      6 |  0.0182230661 |         ideal
    14 |      490 |      6 |  0.000000E+00 |             f
    15 |      520 |      5 |  0.0700527588 |             f
    16 |      550 |      7 |  0.0226815992 |         ide

In [25]:
print(X.shape)
print(F.shape)
samples = np.copy(X)
fitness = np.average(F, axis = 1)
shForces = F[:, 0]
shForces = F[:, 1]

(57, 15)
(57, 2)


In [10]:
plt.figure(figsize=(7, 5))
plt.scatter(F[:, 0], F[:, 1], s=30, facecolors='none', edgecolors='blue')
plt.title("Objective Space")
plt.show()

### Write data to a file

In [26]:
now = str(int(time.time()))
np.savetxt(now + "Population.txt", samples)
np.savetxt(now + "Fitness.txt", fitness)
np.savetxt(now + "ShForces.txt", shForces)
np.savetxt(now + "ShMoments.txt", shMoments)